In [1]:
# ==============================================================================
# Comprehensive Data Analysis Script for Q1 Paper - Clustering Version
# Objective: Discover Smart Farming Profiles via Multi-Algorithm Clustering
# Author: AI Assistant (Adapted for Clustering)
# Version: 1.1 - Corrected Imports
# ==============================================================================

# ------------------------------------------------------------------------------
# Phase 0: Initial Setup, Libraries, and Data Loading
# ------------------------------------------------------------------------------
print("Phase 0: Starting initial setup for Clustering Analysis")

# --- Import essential libraries ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib

# --- Import sklearn tools ---
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score, adjusted_rand_score

# --- Import clustering models ---
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture

# --- General Settings ---
sns.set_theme(style="whitegrid")
plt.rcParams.update({
    'font.size': 7,
    'axes.titlesize': 7,
    'axes.labelsize': 7,
    'xtick.labelsize': 7,
    'ytick.labelsize': 7,
    'legend.fontsize': 7,
    'figure.titlesize': 7,
    'figure.facecolor': 'white'
})
RANDOM_STATE = 42
# The 'label' column will be used for external validation, not for clustering itself
GROUND_TRUTH_COL = 'label'

# --- Define Paths ---
# !!! IMPORTANT: Please update BASE_DIR to your project's root directory !!!
BASE_DIR = '/home/sajad/Sajad_test/Smart_Farming_Data_2024_Clustering/'
FILE_NAME = 'Crop_recommendationV2.csv' # Assuming this is your SF24 dataset file
DATA_PATH = os.path.join(BASE_DIR, 'dataset', FILE_NAME)

RESULTS_PATH = os.path.join(BASE_DIR, 'clustering_results')
EDA_PATH = os.path.join(BASE_DIR, 'EDA')
MODELS_PATH = os.path.join(BASE_DIR, 'models_artifacts')
COMPARISON_PATH = os.path.join(RESULTS_PATH, 'comparison')
BEST_MODEL_ANALYSIS_PATH = os.path.join(RESULTS_PATH, 'best_model_analysis')

paths_to_create = [RESULTS_PATH, EDA_PATH, MODELS_PATH, COMPARISON_PATH, BEST_MODEL_ANALYSIS_PATH]
for path in paths_to_create:
    os.makedirs(path, exist_ok=True)
print("All directories created or verified successfully.")

# --- Helper Functions ---
def save_plot(fig, path, filename, width_cm=20):
    """Helper function to save plots in high-quality formats."""
    width_in = width_cm / 2.54
    aspect_ratio = fig.get_figheight() / fig.get_figwidth()
    fig.set_size_inches(width_in, width_in * aspect_ratio)
    fig.savefig(os.path.join(path, f"{filename}.svg"), format='svg', bbox_inches='tight', dpi=300)
    fig.savefig(os.path.join(path, f"{filename}.pdf"), format='pdf', bbox_inches='tight', dpi=300)
    plt.close(fig)
    print(f"Plot saved: {filename}")

def get_clustering_metrics(data, labels, ground_truth=None):
    """Calculate all relevant clustering evaluation metrics."""
    # Filter out noise points (label == -1) for metric calculation
    valid_indices = labels != -1
    if np.sum(valid_indices) < 2 or len(np.unique(labels[valid_indices])) < 2:
        return {
            'Silhouette Score': np.nan, 'Davies-Bouldin Score': np.nan,
            'Calinski-Harabasz Score': np.nan, 'Adjusted Rand Score': np.nan,
            'Number of Clusters': 0, 'Noise Points': len(labels) - np.sum(valid_indices)
        }
    
    metrics = {
        'Silhouette Score': silhouette_score(data[valid_indices], labels[valid_indices]),
        'Davies-Bouldin Score': davies_bouldin_score(data[valid_indices], labels[valid_indices]),
        'Calinski-Harabasz Score': calinski_harabasz_score(data[valid_indices], labels[valid_indices]),
        'Number of Clusters': len(np.unique(labels[valid_indices])),
        'Noise Points': len(labels) - np.sum(valid_indices)
    }
    if ground_truth is not None:
        metrics['Adjusted Rand Score'] = adjusted_rand_score(ground_truth[valid_indices], labels[valid_indices])
    else:
        metrics['Adjusted Rand Score'] = np.nan
    return metrics

def plot_radar_chart(data, title, save_path, filename):
    """Plots a radar chart for cluster profiling."""
    labels = data.columns
    num_vars = len(labels)
    
    angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
    angles += angles[:1]
    
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
    
    for i, row in data.iterrows():
        values = row.tolist()
        values += values[:1]
        ax.plot(angles, values, label=f'Cluster {i}')
        ax.fill(angles, values, alpha=0.25)
        
    ax.set_yticklabels([])
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels, size=8)
    
    plt.title(title, size=15, color='black', y=1.1)
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
    save_plot(fig, save_path, filename, width_cm=25)

# --- Load Data ---
try:
    df = pd.read_csv(DATA_PATH)
    print("Dataset loaded successfully.")
    print(f"Dataset shape: {df.shape}")
except FileNotFoundError:
    print(f"!!!!!!!!!! CRITICAL ERROR !!!!!!!!!!!\nDataset file not found at:\n{DATA_PATH}")
    exit()

# ------------------------------------------------------------------------------
# Phase 1: Exploratory Data Analysis (EDA)
# ------------------------------------------------------------------------------
print("\nPhase 1: Starting Exploratory Data Analysis (EDA)")
df.describe().to_excel(os.path.join(EDA_PATH, "descriptive_statistics.xlsx"))

numerical_features_all = df.select_dtypes(include=np.number).columns.tolist()
corr_matrix = df[numerical_features_all].corr()
fig, ax = plt.subplots(figsize=(24, 20))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".1f", annot_kws={"size": 8})
ax.set_title("Feature Correlation Matrix", fontsize=16)
save_plot(fig, EDA_PATH, 'correlation_heatmap_all_features', width_cm=40)

# --- PCA for initial visualization ---
print("Performing PCA for initial data visualization...")
temp_df = df.drop(columns=[GROUND_TRUTH_COL], errors='ignore')
numerical_features = temp_df.select_dtypes(include=np.number).columns
scaler = StandardScaler()
df_scaled = scaler.fit_transform(temp_df[numerical_features])

pca = PCA(n_components=2, random_state=RANDOM_STATE)
df_pca = pca.fit_transform(df_scaled)

fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(df_pca[:, 0], df_pca[:, 1], alpha=0.5)
ax.set_title('2D PCA of the Dataset')
ax.set_xlabel(f'Principal Component 1 ({pca.explained_variance_ratio_[0]:.2%})')
ax.set_ylabel(f'Principal Component 2 ({pca.explained_variance_ratio_[1]:.2%})')
save_plot(fig, EDA_PATH, 'pca_visualization_initial', width_cm=20)
print("Exploratory Data Analysis completed.")

# ------------------------------------------------------------------------------
# Phase 1.5: Define Research Scenarios
# ------------------------------------------------------------------------------
print("\nPhase 1.5: Defining research scenarios")
all_features = df.select_dtypes(include=np.number).columns.tolist()
derived_features = ['Temperature-Humidity Index', 'Nutrient Balance Ratio', 'Water Availability Index', 'Photosynthesis Potential', 'Soil Fertility Index']
# Correcting derived feature names if they have different names in the CSV
derived_features_in_df = [f for f in df.columns if any(sub in f for sub in ['THI', 'NBR', 'WAI', 'PP', 'SFI'])]
if not derived_features_in_df: # Fallback if names are different
    derived_features_in_df = derived_features
    
original_features = [f for f in all_features if f not in derived_features_in_df]

scenarios = {
    "With_Derived_Features": all_features,
    "Original_Features_Only": original_features
}
print(f"Scenario 1: {len(scenarios['With_Derived_Features'])} features.")
print(f"Scenario 2: {len(scenarios['Original_Features_Only'])} features.")

# ------------------------------------------------------------------------------
# Phase 2: Data Preprocessing
# ------------------------------------------------------------------------------
print("\nPhase 2: Starting data preprocessing")
if GROUND_TRUTH_COL not in df.columns:
    print(f"Warning: Ground truth column '{GROUND_TRUTH_COL}' not found. External validation will be skipped.")
    ground_truth_labels = None
else:
    ground_truth_labels = df[GROUND_TRUTH_COL]
    df_for_clustering = df.drop(columns=[GROUND_TRUTH_COL])

# We will create a preprocessor inside the loop for each scenario
print("Preprocessing pipeline will be defined per scenario.")

# ------------------------------------------------------------------------------
# Phase 3: Comprehensive Comparison of Clustering Models
# ------------------------------------------------------------------------------
print("\nPhase 3: Starting comprehensive model evaluation and comparison")
models = {
    'KMeans': KMeans(random_state=RANDOM_STATE, n_init=10),
    'Agglomerative': AgglomerativeClustering(),
    'GMM': GaussianMixture(random_state=RANDOM_STATE),
    # DBSCAN parameters (eps, min_samples) need careful tuning. Using defaults as a start.
    'DBSCAN': DBSCAN(eps=1.5, min_samples=5) 
}

all_scenarios_metrics = []
k_range = range(2, 11) # Range of clusters to test for KMeans and GMM

for scenario_name, feature_list in scenarios.items():
    print(f"\n===== EVALUATING SCENARIO: {scenario_name} =====")
    
    X = df[feature_list]
    
    # Define preprocessor for the current set of features
    numerical_features = X.select_dtypes(include=np.number).columns.tolist()
    preprocessor = ColumnTransformer(
        transformers=[('num', StandardScaler(), numerical_features)],
        remainder='passthrough'
    )
    
    X_processed = preprocessor.fit_transform(X)
    
    for model_name, model in models.items():
        print(f"--- Evaluating model: {model_name} ---")
        
        if model_name in ['KMeans', 'GMM', 'Agglomerative']:
            # Find optimal k
            k_metrics = []
            for k in k_range:
                if model_name == 'KMeans':
                    model.set_params(n_clusters=k)
                elif model_name == 'GMM':
                    model.set_params(n_components=k)
                elif model_name == 'Agglomerative':
                    model.set_params(n_clusters=k)
                
                labels = model.fit_predict(X_processed)
                if len(np.unique(labels)) > 1:
                    score = silhouette_score(X_processed, labels)
                    k_metrics.append({'k': k, 'silhouette': score})
            
            if not k_metrics:
                print(f"Could not find a valid clustering for {model_name}. Skipping.")
                continue

            k_metrics_df = pd.DataFrame(k_metrics)
            best_k = k_metrics_df.loc[k_metrics_df['silhouette'].idxmax()]['k']
            print(f"Optimal k for {model_name} found: {int(best_k)}")

            # Plot k-selection graph
            fig, ax = plt.subplots()
            ax.plot(k_metrics_df['k'], k_metrics_df['silhouette'], marker='o')
            ax.set_title(f'Silhouette Score for {model_name} ({scenario_name})')
            ax.set_xlabel('Number of Clusters (k)')
            ax.set_ylabel('Silhouette Score')
            ax.axvline(x=best_k, color='r', linestyle='--', label=f'Optimal k = {int(best_k)}')
            ax.legend()
            save_plot(fig, COMPARISON_PATH, f'k_selection_{model_name}_{scenario_name}', width_cm=15)

            # Rerun with best k
            if model_name == 'KMeans': model.set_params(n_clusters=int(best_k))
            elif model_name == 'GMM': model.set_params(n_components=int(best_k))
            elif model_name == 'Agglomerative': model.set_params(n_clusters=int(best_k))
            
        labels = model.fit_predict(X_processed)
        
        # Save the model and labels
        joblib.dump(model, os.path.join(MODELS_PATH, f"{model_name}_{scenario_name}_model.joblib"))
        np.save(os.path.join(MODELS_PATH, f"{model_name}_{scenario_name}_labels.npy"), labels)

        # Calculate metrics
        metrics = get_clustering_metrics(X_processed, labels, ground_truth=ground_truth_labels)
        metrics['Model'] = model_name
        metrics['Scenario'] = scenario_name
        all_scenarios_metrics.append(metrics)
        
        # Visualize clusters on PCA plot
        fig, ax = plt.subplots(figsize=(10, 8))
        unique_labels = np.unique(labels)
        palette = sns.color_palette("hsv", len(unique_labels))
        for i, label in enumerate(unique_labels):
            if label == -1:
                # Plot noise points in black
                cluster_color = 'k'
                point_label = 'Noise'
            else:
                cluster_color = palette[i]
                point_label = f'Cluster {label}'
            
            ax.scatter(df_pca[labels == label, 0], df_pca[labels == label, 1],
                       c=[cluster_color], label=point_label, alpha=0.7, s=30)
        
        ax.set_title(f'Clusters by {model_name} on PCA ({scenario_name})')
        ax.set_xlabel(f'Principal Component 1')
        ax.set_ylabel(f'Principal Component 2')
        ax.legend()
        save_plot(fig, COMPARISON_PATH, f'pca_clusters_{model_name}_{scenario_name}', width_cm=20)

results_df = pd.DataFrame(all_scenarios_metrics).sort_values(by='Silhouette Score', ascending=False)
results_df.to_excel(os.path.join(COMPARISON_PATH, "all_models_metrics_comparison.xlsx"), index=False)
print("\nComprehensive model comparison completed. Results saved.")
print(results_df)

# ------------------------------------------------------------------------------
# Phase 4: Best Model Selection and Cluster Profiling
# ------------------------------------------------------------------------------
print(f"\nPhase 4: Starting selection of best model and cluster profiling")

if results_df.empty:
    print("No valid clustering results found. Exiting.")
    exit()

best_model_info = results_df.iloc[0]
best_model_name = best_model_info['Model']
best_scenario_name = best_model_info['Scenario']
print(f"Best performing model identified: {best_model_name} under scenario '{best_scenario_name}'")
print(f"Metrics: Silhouette={best_model_info['Silhouette Score']:.3f}, Davies-Bouldin={best_model_info['Davies-Bouldin Score']:.3f}")

# Load the best model's labels
best_labels = np.load(os.path.join(MODELS_PATH, f"{best_model_name}_{best_scenario_name}_labels.npy"))

# Create a profiling dataframe
profile_df = df.copy()
profile_df['Cluster'] = best_labels

# Exclude noise points from profiling if any
profile_df = profile_df[profile_df['Cluster'] != -1]

# Calculate cluster profiles (mean of each feature per cluster)
cluster_profiles = profile_df.groupby('Cluster').mean(numeric_only=True)
cluster_profiles.to_excel(os.path.join(BEST_MODEL_ANALYSIS_PATH, "cluster_profiles_mean.xlsx"))

# Calculate cluster sizes and distribution of 'label'
cluster_sizes = profile_df['Cluster'].value_counts().sort_index()
print("\nCluster Sizes:")
print(cluster_sizes)

if ground_truth_labels is not None:
    label_distribution = pd.crosstab(profile_df['Cluster'], profile_df[GROUND_TRUTH_COL])
    label_distribution.to_excel(os.path.join(BEST_MODEL_ANALYSIS_PATH, "cluster_label_distribution.xlsx"))
    print("\nDistribution of Crop Types per Cluster:")
    print(label_distribution)

print("Cluster profiling completed.")

# ------------------------------------------------------------------------------
# Phase 5: In-depth Analysis & Visualization of Best Model's Profiles
# ------------------------------------------------------------------------------
print(f"\nPhase 5: Starting in-depth analysis and visualization of profiles")

# --- Radar Chart for Cluster Profiles ---
# Select key features for the radar chart for better readability
key_profile_features = [
    'N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'soil_moisture',
    'sunlight_exposure', 'organic_matter', 'pest_pressure', 'fertilizer_usage',
    'THI', 'NBR', 'WAI', 'PP', 'SFI'
]
# Use correct names from dataframe
key_profile_features_in_df = [f for f in cluster_profiles.columns if f in key_profile_features or any(sub in f for sub in ['THI', 'NBR', 'WAI', 'PP', 'SFI'])]

# Normalize data for radar chart (0-1 scaling)
radar_data = cluster_profiles[key_profile_features_in_df]
radar_data_normalized = (radar_data - radar_data.min()) / (radar_data.max() - radar_data.min())

plot_radar_chart(
    radar_data_normalized,
    f'Normalized Profiles of Clusters (Model: {best_model_name})',
    BEST_MODEL_ANALYSIS_PATH,
    'cluster_profiles_radar_chart'
)

# --- Boxplots for Key Features per Cluster ---
print("Creating boxplots for key features across clusters...")
features_for_boxplot = ['temperature', 'humidity', 'soil_moisture', 'SFI', 'WAI', 'pest_pressure']
# Use correct names from dataframe
features_for_boxplot_in_df = [f for f in profile_df.columns if f in features_for_boxplot or any(sub in f for sub in ['SFI', 'WAI'])]


n_features = len(features_for_boxplot_in_df)
n_cols = 3
n_rows = (n_features + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 6, n_rows * 5), sharex=True)
axes = axes.flatten()

for i, feature in enumerate(features_for_boxplot_in_df):
    sns.boxplot(data=profile_df, x='Cluster', y=feature, ax=axes[i], palette='viridis')
    axes[i].set_title(f'Distribution of {feature} by Cluster')

# Hide unused subplots
for i in range(n_features, len(axes)):
    axes[i].set_visible(False)

fig.tight_layout(pad=3.0)
save_plot(fig, BEST_MODEL_ANALYSIS_PATH, 'key_features_boxplot_by_cluster', width_cm=30)

print("\n" + "="*80 + "\nClustering script executed successfully! All results and artifacts are saved.\n" + "="*80)

Phase 0: Starting initial setup for Clustering Analysis
All directories created or verified successfully.
Dataset loaded successfully.
Dataset shape: (2200, 23)

Phase 1: Starting Exploratory Data Analysis (EDA)
Plot saved: correlation_heatmap_all_features
Performing PCA for initial data visualization...
Plot saved: pca_visualization_initial
Exploratory Data Analysis completed.

Phase 1.5: Defining research scenarios
Scenario 1: 22 features.
Scenario 2: 22 features.

Phase 2: Starting data preprocessing
Preprocessing pipeline will be defined per scenario.

Phase 3: Starting comprehensive model evaluation and comparison

===== EVALUATING SCENARIO: With_Derived_Features =====
--- Evaluating model: KMeans ---
Optimal k for KMeans found: 2
Plot saved: k_selection_KMeans_With_Derived_Features
Plot saved: pca_clusters_KMeans_With_Derived_Features
--- Evaluating model: Agglomerative ---
Optimal k for Agglomerative found: 2
Plot saved: k_selection_Agglomerative_With_Derived_Features
Plot saved

/tmp/ipykernel_754086/3626803.py:390: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=profile_df, x='Cluster', y=feature, ax=axes[i], palette='viridis')
/tmp/ipykernel_754086/3626803.py:390: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=profile_df, x='Cluster', y=feature, ax=axes[i], palette='viridis')
/tmp/ipykernel_754086/3626803.py:390: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=profile_df, x='Cluster', y=feature, ax=axes[i], palette='viridis')
/tmp/ipykernel_754086/3626803.py:390: FutureWarning: 

Passing `palette` withou

Plot saved: key_features_boxplot_by_cluster

Clustering script executed successfully! All results and artifacts are saved.
